#### Importer les modules
En général, je préfère importer les modules au haut de mes notebooks.

In [ ]:
!pip install lets-plot==4.1 polars==0.19
import polars as pl
from lets_plot import *
LetsPlot.setup_html() 
import numpy as np
from scipy.stats import binom, poisson, uniform, norm

# Statistiques

Aux chapitres précédents, nous avons vu comment visualiser, organiser et manipuler des tableaux de données. La statistique est une collection de disciplines liées à la collecte, l’organisation, l'analyse, l'interprétation et la présentation de données.

Dans [*Principles and procedures of statistics: A biometrical approach*](https://www.amazon.com/Principles-Procedures-Statistics-Biometrical-Approach/dp/0070610282), Steel, Torie et Dickey (1997) définissent les statistiques ainsi :

> Les statistiques forment la science, pure et appliquée, de la création, du développement, et de l'application de techniques par lesquelles l'incertitude de l'induction inférentielle peut être évaluée. (ma traduction)

Alors que l'**inférence** consiste à généraliser des échantillons à l'ensemble d'une population, l'**induction** est un type de raisonnement qui permet de généraliser des observations sous forme de théories. En d'autres mots, les statistiques permettent d'évaluer l'incertitude sur des processus, de passer par inférence de l'échantillon à la population, puis par induction de passer de cette représentation d'une population en lois générales la concernant.

La définition de Whitlock et Schuluter (2015), dans [The Analysis of Biological Data](http://whitlockschluter.zoology.ubc.ca/), est plus simple et n'insiste que sur l'inférence:

> La statistique est l’étude des méthodes pour mesurer des aspects de populations à partir d’échantillons et pour quantifier l'incertitude des mesures. (ma traduction)

Les statistiques consistent à *faire du sens* (anglicisme assumé) avec des observations dans l'objectif de répondre à une question que vous aurez formulée clairement, préalablement à votre expérience.

<blockquote class="twitter-tweet" data-lang="fr"><p lang="en" dir="ltr">The more time I spend as The Statistician in the room, the more I think the best skill you can cultivate is the ability to remain calm and repeatedly ask &quot;What question are you trying to answer?&quot;</p>&mdash; Bryan Howie (@bryan_howie) <a href="https://twitter.com/bryan_howie/status/1073054519808876544?ref_src=twsrc%5Etfw">13 décembre 2018</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

Le flux de travail conventionnel en statistiques consiste à collecter des échantillons, transformer (prétraiter) les données, effectuer des tests, analyser les résultats, les interpréter et les visualiser. Mais avant tout: **se demander à quelle question veut-on répondre**.

Cette section à elle seule est trop courte pour permettre d'intégrer toutes les connaissances nécessaires à une utilisation raisonnée des statistiques, mais fournit les bases pour aller plus loin. Notez que les erreurs d'interprétation statistiques sont courantes et la consultation de spécialistes n'est souvent pas un luxe. Mais bien que les statistiques soient complexes, *la plupart des opérations statistiques peuvent être effectuées sans l'assistance de statisticien.nes*... à condition de comprendre suffisamment les concepts utilisés.

Dans cette section, nous verrons comment répondre correctement à une question valide et adéquate avec l'aide d'outils de calcul scientifique. Nous couvrirons les notions de bases des distributions et des variables aléatoires qui nous permettront d'effectuer des tests statistiques communs avec Python. Nous couvrirons aussi les erreurs communément commises en recherche académique et les moyens simples de les éviter. Nous utiliserons évidemment les modules *Polars* et *Lets-Plot*. J'introduirai les modules *Scipy* et *Statsmodels*.

Je vais d'abord rapidement présenter quelques notions importantes en statistiques : populations et échantillons, variables, probabilités et distributions. Puis nous allons effectuer des tests d'hypothèse univariés et détailler la notion de plus en plus controversée de *p-value*. Je vais m'attarder plus longuement aux modèles linéaires généralisés, qui fournissent une trousse d'analyse polyvalente en analyse multivariée.


## Populations et échantillons

Le principe d'inférence consiste à généraliser des conclusions à l'échelle d'une population à partir d'échantillons issus de cette population. Alors qu'une **population** contient tous les éléments étudiés, un **échantillon** d'une population est une observation unique. Une expérience bien conçue fera en sorte que les échantillons soient représentatifs de la population qui, la plupart du temps, ne peut être observée entièrement pour des raisons pratiques.

Je ne couvre pas les principes d'expérimentation et les dispositifs expérimentaux servant de base à la conception d'une bonne méthodologie. À ce titre, je recommande le livre *Principes d'expérimentation: planification des expériences et analyse de leurs résultats* de Pierre Dagnelie (2012), [disponible en ligne en format PDF](http://www.dagnelie.be/docpdf/ex2012.pdf). Un bon aperçu des dispositifs expérimentaux est aussi présenté dans [*Introductory Statistics with R*](https://www.springer.com/us/book/9780387790534), de Peter Dalgaard (2008).

Une population est échantillonnée pour induire des **paramètres**: un rendement typique dans des conditions météorologiques, édaphiques et managériales données, la masse typique des faucons pèlerins, mâles et femelles, le microbiome typique d'un sol géotechnique, agricole ou forestier, etc. Une **statistique** est une estimation d'un paramètre calculée à partir des données, par exemple une moyenne et un écart-type, ou un intercept et une pente.

Par exemple, la moyenne et l'écart-type d'une population sont estimés par les moyennes et écarts-types calculés sur les données issues de l'échantillonnage.

Chaque paramètre est lié à une perspective que l'on désire connaître chez une population. Ces angles d'observations sont les **variables**.

## Les variables

Nous avons abordé à la section 2 la notion de *variable* par l'intermédiaire d'une donnée. Une variable est l'observation d'une caractéristique décrivant un échantillon. Si la caractéristique varie d'un échantillon à un autre sans que vous en expliquiez la raison (i.e. si identifier la source de la variabilité ne fait pas partie de votre expérience), on parlera de variable aléatoire. Même le hasard est régi par certaines lois: ce qui est aléatoire dans une variable peut être décrit par des **lois de probabilité**, que nous verrons plus bas.

Il existe plusieurs types de variables, qui se regroupe en deux grandes catégories: les **variables quantitatives** et les **variables qualitatives**.

### Variables quantitatives

Ces variables peuvent être continues dans un espace échantillonnal réel, ou bien discrètes dans un espace échantillonnal ne considérant que des valeurs fixes. Bien que les valeurs fixes des distributions discrètes ne soient pas toujours des valeurs entières, c'est bien souvent le cas en biostatistiques comme en démographie, où les décomptes d'individus sont souvent présents (et où la notion de fraction d'individus n'est pas acceptée).

### Variables qualitatives

On exprime parfois qu'une variable qualitative est une variable impossible à mesurer numériquement : une couleur, l'appartenance à espèce ou à une série de sol. Pourtant, dans bien des cas, les variables qualitatives peuvent être encodées en variables quantitatives. Par exemple, on peut accoler des pourcentages de sable, limon et argile à un loam sableux, qui autrement est décrit par la classe texturale d'un sol. Pour une couleur, on peut lui associer une longueur d'onde ou des pourcentages de rouge, vert et bleu, ainsi qu'un ton. En ce qui a trait aux variables ordonnées, il est possible de supposer un étalement. Par exemple, une variable d'intensité faible-moyenne-forte peut être transformée linéairement en valeurs quantitatives -1, 0 et 1. Attention toutefois, l'étalement peut parfois être quadratique ou logarithmique. Quant aux catégories difficilement transformables en quantités, on pourra passer par l'**encodage catégoriel**, souvent appelé *dummyfication*, que nous verrons plus loin. L'analyse qualitative consiste en l'analyse de verbatims, essentiellement utile en sciences sociales : nous n'en n'aurons pas besoin ici. Nous considérerons les variables qualitatives comme des variables quantitatives qui n'ont pas subi de prétraitement.


## Les probabilités

Une probabilité est la vraisemblance qu'un évènement se réalise chez un échantillon. Les probabilités forment le cadre des systèmes stochastiques, c'est-à-dire des systèmes trop complexes pour en connaître exactement les aboutissants, auxquels on attribue une part de hasard. Ces systèmes sont prédominants dans les processus vivants.

On peut dégager deux perspectives sur les probabilités: l'une passe par une interprétation fréquentielle, l'autre bayésienne.

- L'approche fréquentielle teste si les données concordent avec un modèle du réel. Il s'agit de l'interprétation communément utilisée.
- L'approche bayésienne évalue la probabilité que le modèle soit réel.

Techniquement, les statistiques fréquentielles sont davantage analytiques, alors que les statistiques bayésiennes sont davantage numériques. Je ne vais couvrir que les statistiques fréquentielles, qui sont plus communes est plus faciles à appréhender. En pratique, je préfère tiliser les statistiques bayésiennes, qui demandent néanmoins une bonne puissance de calcul.

## Les distributions

Une variable aléatoire peut prendre des valeurs selon des modèles de distribution des probabilités. Une distribution est une fonction mathématique décrivant la probabilité d'observer des évènements. Ces évènements peuvent être des valeurs continues, des nombres entiers, des catégories, des valeurs booléennes (Vrai/Faux), etc. Dépendamment du type de valeur et des observations obtenues, on peut associer des variables à différentes lois de probabilité. L'aire sous la courbe d'une distribution de probabilité est toujours égale à 1.

En statistiques inférentielles, les distributions sont les modèles, comprenant certains paramètres comme la moyenne et la variance pour les distributions normales, à partir desquelles les données sont générées.

Il existe deux grandes familles de distribution: **discrètes** et **continues**. Les distributions discrètes sont contraintes à des valeurs prédéfinies (finies ou infinies), alors que les distributions continues prennent nécessairement un nombre infini de valeur, dont la probabilité ne peut pas être évaluée ponctuellement, mais sur un intervalle.

L'**espérance** mathématique est une fonction de tendance centrale, souvent décrite par un paramètre. Il s'agit de la moyenne d'une population pour une distribution normale. La **variance**, quant à elle, décrit la variabilité d'une population, i.e. son étalement autour de l'espérance. Pour une distribution normale, la variance d'une population est aussi appelée variance, représentée aussi par l'écart-type (égal à la racine carrée de la variance).

Pour éviter de trop m'étaler, je ne présente que la distribution de Poisson et la distribution normale, mais il en existe [vraiment beaucoup](https://www.pymc.io/projects/docs/en/stable/api/distributions.html).

### Distribution de Poisson

Les distributions de Poisson décrivent ce qui tient du décompte (avec un P majuscule, à moins que vous comptiez des poissons). Il peut s'agir du nombre de grenouilles traversant une rue quotidiennement, du nombre de plants d'asclépiades se trouvant sur une terre cultivée, ou du nombre d’évènements de précipitation au mois de juin, etc. La distribution de Poisson n'a qu'un seul paramètre, λ, qui décrit tant la moyenne des décomptes.

Par exemple, en un mois de 30 jours, et une moyenne de 8 évènements de précipitation pour ce mois, on obtient la distribution suivante.


In [2]:
x = np.arange(0, 31)
prob = poisson(mu=8).pmf(x)
df_poisson = pl.DataFrame({'x':x, 'prob': prob})
ggplot(data=df_poisson, mapping=aes(x='x', y='prob')) + geom_lollipop()

Notez bien que, par réflexe, je crée mes graphiques avec *Lets-Plot*, non pas avec l'outil de *Noteable*.

### Distribution normale

La plus répandue de ces lois est probablement la loi normale, parfois nommée loi gaussienne et plus rarement loi laplacienne. Il s'agit de la distribution classique en forme de cloche.

La loi normale est décrite par une moyenne, qui désigne la tendance centrale, et une variance, qui désigne l'étalement des probabilités autour de la moyenne. La racine carrée de la variance est l'écart-type.

Les distributions de mesures exclusivement positives (comme le poids ou la taille) sont parfois avantageusement approximées par une loi **log-normale**, qui est une loi normale sur le logarithme des valeurs: la moyenne d'une loi log-normale est la moyenne géométrique.

In [3]:
n_points = 100
x = np.linspace(-10, 10, n_points)
prob1 = norm(-4, 2).pdf(x)
prob2 = norm(0, 4).pdf(x)
prob3 = norm(4, 6).pdf(x)
df_norm = pl.DataFrame({
    'x':np.hstack([x, x, x]),
    'prob': np.hstack([prob1, prob2, prob3]),
    'catégorie': np.hstack(
        [np.repeat('A', n_points), np.repeat('B', n_points), np.repeat('C', n_points)]
    )
})
ggplot(df_norm, aes(x='x', y='prob')) + geom_line(aes(color='catégorie'))

Quelle est la probabilité de retrouver une valeur dans l’intervalle entre -1 et 2 sur la courbe de catégorie B?

In [4]:
norm(0, 4).cdf(2) - norm(0, 4).cdf(-1) 

0.2901687869569368

On pourrait aussi s'intéresser à l'intervalle de confiance sur la moyenne. En effet, la moyenne suit aussi une distribution normale, dont la tendance centrale est la moyenne de la distribution, et dont l'écart-type est noté *erreur standard*. On calcule cette erreur en divisant la variance par le nombre d'observation, ou en divisant l'écart-type par la racine carrée du nombre d'observations. Ainsi, pour 10 échantillons:

In [5]:
alpha = 0.05
moyenne = 0
ecart_type = 1
n_echantillon = 10
norm.interval(
    1-alpha/2,
    loc = moyenne,
    scale = ecart_type / np.sqrt(n_echantillon)
)

(-0.708793777294559, 0.7087937772945595)

## Les modèles linéaires généralisés

Il existe une panoplie de tests statistiques, et l'on peut facilement se perdre dans les détails et suppositions qu'ils sous-entendent. À mon expérience, la plupart des besoins en sciences peuvent être répondus par des modèles linéaires généralisés.

- **Modèle**. Une maquette mathématique qui représente ce qui s'est passé dans le contexte dans lequel les données ont été recueillies.
- **Linéaire**. Une combinaison linéaire de variable. Un modèle quadratique est linéaire dans le sens qu'il est une combinaison linéaire d'une variable et de sa représentation transformée au carré. Un phénomène qui semble non-linéaire à cause d'une courbure dans sa représentation graphique peut ainsi être exprimé de manière linéaire, mais sur des variables transformées, ou sur des variables non transformées mais e incluant des variables complémentaires, ou à l'extrême être une combinaison linéaire de fonctions non-linéaires. À ce titre, les modèles additifs généralisés (*generalized additive model, GAM*, voir [le cours de Noam Ross en langage R ](noamross.github.io/gams-in-r-course/), certaines implémentations en Python existent) sont très intéressants, mais s'écartent du sujet.
- **Généralisé**. La (ou les) sortie(s) du modèle peuvent être décrits par différentes lois de probabilité. Un décompte du nombre de passages d'automobile sur une route pas s'approxime mieux par une loi de Poisson qu'une loi normale. Un modèle linéaire est généralisé si on tient compte du type de distribution ayant potentiellement généré les donnéerimentales.

Le module [*statsmodels*](statsmodels.org) facilite la composition d'un GLM en offrant la possibilité d'utiliser une interface par formule similaire à ce que l'on retrouve en langage R. Cette interface se présente, par exemple, comme suit.

`y ~ v1 + v2 + v3 * v4`

Le `y` est la sortie du modèle, aussi appelée variable dépendante ou endogène. Le `~` se lit non pas comme "égal à", mais comme "en fonction de". Les variables additionnées montrent un effet additif. Chacune des variables sera associé à un coefficient dans le modèle. L'astérisque entre `v3` et `v4` signifie que l'on désire entrer les effets additifs de v3 et v4, mais en ajoutant une interaction, c'est-à-dire une nouvelle variable constituée de la multiplication des variables `v3` et `v4`. Si l'on désirait ne garder que l'interaction entre `v3` et `v4`, nous pourrions utiliser un `:` au lieu de l'`*`, de sorte que l'équation précédente est l'équivalent de `y ~ v1 + v2 + v3 + v4 + v3:v4`. L'intercept du modèle est ajouté implicitement. Si on désire un modèle sans intercept, il faudra ajouter un `0 +` à droite du `~`. Davantage d'info sur l'interface formule peut être trouvée dans la documentation du module [*patsy*](https://patsy.readthedocs.io/en/latest/formulas.html), sur lequel repose *statsmodels*.



Les données `mtcars` sont souvent utilisées pour donner des exemples en stats.

In [6]:
mtcars = pl.read_csv('https://gist.githubusercontent.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv')
mtcars

model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
str,f64,i64,f64,i64,f64,f64,f64,i64,i64,i64,i64
"""Mazda RX4""",21.0,6,160.0,110,3.9,2.62,16.46,0,1,4,4
"""Mazda RX4 Wag""",21.0,6,160.0,110,3.9,2.875,17.02,0,1,4,4
"""Datsun 710""",22.8,4,108.0,93,3.85,2.32,18.61,1,1,4,1
"""Hornet 4 Drive…",21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
"""Hornet Sportab…",18.7,8,360.0,175,3.15,3.44,17.02,0,0,3,2
"""Valiant""",18.1,6,225.0,105,2.76,3.46,20.22,1,0,3,1
"""Duster 360""",14.3,8,360.0,245,3.21,3.57,15.84,0,0,3,4
"""Merc 240D""",24.4,4,146.7,62,3.69,3.19,20.0,1,0,4,2
"""Merc 230""",22.8,4,140.8,95,3.92,3.15,22.9,1,0,4,2


Nous nous intéresserons aux colonnes suivantes.

- `mpg`. Miles per gallon, la distance parcourue avec un gallon d'essence.
- `cyl`. Le nombre de pistons dans le moteur.
- `disp`. Le volume total des cylindres du moteur.
- `hp`. La puissance du moteur.
- `wt`. Le poids du véhicule.
- `gear`. Le nombre de ratio d'engrenages.
- `vs`. Si la forme des cylindres est en v (`vs = 1`) ou droite (`vs = 0`).

Question posée. Lesquelles parmi ces variables influencent le plus la consommation d'essence ?

On peut y répondre avec un modèle linéaire avec sortie normale.

In [7]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

mtcars_model = smf.glm(
    formula = "mpg ~ cyl + disp + hp + wt + gear + vs",
    data=mtcars,
    family=sm.families.Gaussian()
).fit()

mtcars_model.summary()

/Users/essi/mambaforge/envs/dl/lib/python3.10/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    mpg   No. Observations:                   32
Model:                            GLM   Df Residuals:                       25
Model Family:                Gaussian   Df Model:                            6
Link Function:               identity   Scale:                          6.6662
Method:                          IRLS   Log-Likelihood:                -71.809
Date:                Fri, 17 Nov 2023   Deviance:                       166.66
Time:                        09:29:57   Pearson chi2:                     167.
No. Iterations:                     3   Pseudo R-squ. (CS):             0.9892
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     35.5081      7.715      4.602      0.000      20.387      50.630
cyl           -0.9306      0.872     -1.067      0.286      -2.640       0.779
disp           0.0146      0.013      1.149      0.250      -0.010       0.039
hp            -0.0287      0.017     -1.687      0.092      -0.062       0.005
wt            -3.7746      1.077     -3.505      0.000      -5.885      -1.664
gear           0.8235      1.115      0.738      0.460      -1.363       3.010
vs             0.6751      1.733      0.390      0.697      -2.721       4.071
==============================================================================
"""

L'intercept est la valeur, dans les unités de la variable réponse (miles par gallon), à laquelle on s'attendrait si toutes les autres variables étaient nulles. Cette valeur n'indique pas grand-chose, étant donné qu'elle correspond entre autres à une voiture dont le poids est nul... aurait du sens si les variables avaient été centrées, de sorte qu'il aurait exprimé le MPG alors que toutes les variables sont à leur moyenne. Le modèle nous informe que la distance de base par gallon est de 35.5 avec un intervalle de confiance à 95% entre 20.4 et 50.6. La p-value est bien sûr très faible: rappelez-vous bien que la p-value est la probabilité que le tableau de résultats ait été généré par le modèle que nous avons décrit et dont l'intercept serait nul.

Les coefficients des autres variables nous indiquent comment l'augmentation d'une unité affecte le MPG. Si le poids du véhicule augmente d'une unité, ce qui équivaut à 1000 lbs, le MPG a diminué de -3.8. La valeur du coefficient n'indique toutefois pas l'importance dans le modèle, puisqu'elle dépend des unités utilisées pour exprimer les Si le

Mais comme on a en discuté, n'est pas nécessairement tirée d'une distribution normale. Reprenons nos données sur les manchots, et posons la question suivante.

> Quelles variables les dimensions du du bec permettent d'expliquer l'espèce de manchot?

 Mais d'abord, il faut s'assurer que la colonne `species` soit exprimée en chiffres.

In [8]:
penguins = pl.read_csv('data/penguins.csv', null_values='NA').drop_nulls()
unique_species = penguins.select('species').unique()
species_no = np.arange(unique_species.shape[0])
unique_species = unique_species.with_columns(pl.lit(species_no).alias("species_no"))
penguins = penguins.join(unique_species, on='species', how='left')

In [9]:
penguin_model = smf.mnlogit(
    formula = "species_no ~ bill_length_mm + bill_depth_mm",
    data=penguins
).fit()
penguin_model.summary()

Optimization terminated successfully.
         Current function value: 0.071740
         Iterations 12


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:             species_no   No. Observations:                  333
Model:                        MNLogit   Df Residuals:                      327
Method:                           MLE   Df Model:                            4
Date:                Fri, 17 Nov 2023   Pseudo R-squ.:                  0.9319
Time:                        09:30:07   Log-Likelihood:                -23.889
converged:                       True   LL-Null:                       -350.86
Covariance Type:            nonrobust   LLR p-value:                3.259e-140
==================================================================================
  species_no=1       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         24.3847     13.527      1.803      0.071      -2.127      50.897
bill_length_mm    -2.2063      0.686     -3.218      0.001      -3.550      -0.863
bill_depth_mm      3.9759      1.486      2.676      0.007       1.063       6.888
----------------------------------------------------------------------------------
  species_no=2       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         49.4000     14.922      3.310      0.001      20.153      78.647
bill_length_mm     0.4865      0.157      3.102      0.002       0.179       0.794
bill_depth_mm     -4.3454      1.037     -4.192      0.000      -6.377      -2.314
==================================================================================
"""

Les résultats demanderaient qu'on s'y attarde longuement. Les coefficients montrent l'ampleur de l'écart entre l'espèce N et l'espèce de référence (Adelie, étiquetée par un `0`). Sans considérer le bec, les différences entre l'espèce `1` (Gentoo) sont l'Adelie hétérogène (p-value, `P>|z| = 0.214`), mais les dimensions du bec y contribuent significativement (`P>|z| > 0.000`). Idem pour l'espèce `2` (Chinstrap).

Ce modèle est évidemment très simplifié. On aurait pu ajouter d'autres dimensions, la masse considérant le sexe, l'île, etc (à vrai dire, j'ai essayé, mais sans succès). Je voulais surtout montrer que la sortie d'un modèle linéaire généralisée pouvait être autre qu'une  continue, pour laquelle on suppose une distribution normale.

Les tests d’hypothèse évaluent des effets statistiques (qui ne sont pas nécessairement des effets de causalité). L’effet à évaluer peut être celui d’un traitement, d’indicateurs météorologiques, de caractéristiques d'un sol, etc. Par convention, l’hypothèse nulle (écrite H<sub>0</sub>) est l’hypothèse qu’il n’y ait pas d’effet (c’est l’hypothèse de l’avocat du diable 😈) à l’échelle de la population (et non pas à l’échelle de l’échantillon). À l’inverse, l’hypothèse alternative (écrite H<sub>1</sub>) est l’hypothèse qu’il y ait un effet à l’échelle de la population.